In [ ]:
# 將 nsta24 轉成 dictionary
def get_StationInfo():
    s = []
    with open("/mnt/nas3/weiwei/CWBDataset/nsta/nsta24.dat") as f:
        for line in f.readlines():
            l = line.strip().split()
           
            d = {}
            d['station'] = l[0]
            d['lon'] = float(l[1])
            d['lat'] = float(l[2])
            d['elevation'] = float(l[3])
            d['location'] = '0' + l[5]
            d['institute'] = l[6]
            d['network'] = l[7]
            d['instrument'] = l[8]
            d['E'] = float(l[9])
            d['N'] = float(l[10])
            d['Z'] = float(l[11])
            d['start'] = l[-2]
            d['end'] = l[-1]
            s.append(d)

    return s

In [2]:
# 根據 scnl 四個 attributes 以及資料時間來取得對應測站儀器的 "轉換量" 與 "經緯度, 高程"
def get_factor(station, instrument, network, location, starttime, stationInfo, station_list, sensor, usage, record_inst):
    
    # starttime: 2021-01-01T04:57:20.000000Z
    
    a_time = starttime[:4] + starttime[5:7] + starttime[8:10]
    st_time = int(a_time)
    
    # 尋找相對應的測站
    for idx, s in enumerate(stationInfo):
        starttime = int(s['start'])
        end = int(s['end'])

        if (st_time >= starttime and st_time <= end):
            if (station == s['station'] and instrument == s['instrument'] and network == s['network'] and location == s['location']):
                return [s['Z'], s["N"], s['E']], s['lon'], s['lat'], s['elevation']
    
    # 從 nsta24 沒找到，再從另外一張表去找
    station_code = find_station_code(station_list, station, record_inst)
    
    factor, lon, lat, elevation = find_coord_and_factor(usage, sensor, station_code, st_time, instrument)
    
    if factor is not None:
        return factor, lon, lat, elevation
    else:
        # 啥都沒對到
        return None, None, None, None

In [3]:
# 將 json 紀錄的時間格式轉成 datetime 格式
def toTime(time):
    year, month, day = int(time[:4]), int(time[5:7]), int(time[8:10])
    hour, minute, second = int(time[11:13]), int(time[14:16]), int(time[17:19])

    if len(time) > 19:
        microsec = int(time[20:-1])
        return datetime.datetime(year, month, day, hour, minute, second, microsec)
    else:
        return datetime.datetime(year, month, day, hour, minute, second)

In [4]:
# 根據兩個時間來判斷之間差距多少個 sample
def time_diff_sample(time1, time2, sampling_rate):
    time1, time2 = toTime(time1), toTime(time2)
    
    diff = time2-time1
    diff_sec, diff_msec = diff.seconds, diff.microseconds
    
    sample = int((diff_sec+diff_msec/1e+5)*sampling_rate)
    
    return sample

In [5]:
# 讀取 .txt 波型
def readWave(path):
    with open(path, 'r')as f:
        tmp = f.readlines()

    z, n, e = [], [], []
    for i in tmp[1:]:
        value = i.split()

        z.append(value[0])
        n.append(value[1])
        e.append(value[2])
        
    z, n, e = np.array(z), np.array(n), np.array(e)
    z, n, e = z.astype(float), n.astype(float), e.astype(float)
    
    return z, n, e

In [12]:
# 計算 Z 軸 snr (db: 10*log10(snr))
def Zsnr_in_db(z, p_arr):
    # 物理量修正 (去平均)
    z = z - np.mean(z)
        
    if p_arr < 0:
        return None
    
    if (p_arr + 150) <= z.shape[0]-1:
        signal = z[p_arr:p_arr+150]
    else:
        signal = z[p_arr:]
    signal=np.mean(np.power(signal,2))
    
    if p_arr >= 500:
        noise = z[p_arr-500:p_arr]
    else:
        noise = z[:p_arr]
    noise=np.mean(np.power(noise,2))
    
    ratio=signal/noise
    
    return 10 * np.log10(ratio)

In [7]:
# 計算三軸合成震波的 snr
def snr_in_db(z, n, e, p_arr):
    # 物理量修正 (去平均)
    z = z - np.mean(z)
    n = n - np.mean(n)
    e = e - np.mean(e)
    
    acc = np.sqrt(z**2+n**2+e**2)
    
    if p_arr < 0:
        return None
    
    if (p_arr + 150) <= z.shape[0]-1:
        signal = acc[p_arr:p_arr+150]
    else:
        signal = acc[p_arr:]
    signal=np.mean(np.power(signal,2))
    
    if p_arr >= 500:
        noise = acc[p_arr-500:p_arr]
    else:
        noise = acc[:p_arr]
    noise=np.mean(np.power(noise,2))
    
    ratio=signal/noise
    
    return 10 * np.log10(ratio)

In [ ]:
# 計算三軸與 Z 軸的 pga: %g, gal(cm/s2), 與 Z 軸的 pga
def calc_pga(z, n, e, sample_rate, instrument):
    # 物理量修正 (去平均)
    z = z - np.mean(z)
    n = n - np.mean(n)
    e = e - np.mean(e)
    
    if instrument == 'SP' or instrument == 'BB':
        z, n, e = v_to_a(z, n, e, sample_rate)
        
    # 10Hz low pass filter
    z = lowpass(z, 10, sample_rate)
    n = lowpass(n, 10, sample_rate)
    e = lowpass(e, 10, sample_rate)
    
    # 合成震波
    acc = np.sqrt(n**2+e**2)
    
    pga = np.max(acc)  # gal
    z_pga = np.max(z)  # gal
    pga_g = pga * (1/9.81)  # %g
        
    return pga, pga_g, z_pga

In [ ]:
# 計算三軸與 Z 軸的 pgv: cm/s
def calc_pgv(z, n, e, sample_rate, instrument):
    # 物理量修正 (去平均)
    z = z - np.mean(z)
    n = n - np.mean(n)
    e = e - np.mean(e)
    
    # 檢查波形是 速度 or 加速度
    if instrument == 'FBA':
        z, n, e = a_to_v(z, n, e, sample_rate)
 
    # 0.075Hz high pass filter
    z = highpass(z, 0.075, sample_rate)
    n = highpass(n, 0.075, sample_rate)
    e = highpass(e, 0.075, sample_rate)
        
    # 合成震波
    acc = np.sqrt(n**2+e**2)
    
    pgv = np.max(acc)  # gal
    z_pgv = np.max(z)  # gal
    
    return pgv, z_pgv

In [9]:
# high pass filter
def highpass(data, freq, df, corners=4, zerophase=False):
    fe = 0.5 * df
    f = freq / fe
    
    z, p, k = iirfilter(corners, f, btype='highpass', ftype='butter', output='zpk')
    sos = zpk2sos(z, p, k)
    if zerophase:
        firstpass = sosfilt(sos, data)
        return sosfilt(sos, firstpass[::-1])[::-1]
    else:
        return sosfilt(sos, data)

In [ ]:
# low pass filter
def lowpass(data, freq, df, corners=4, zerophase=False):
    fe = 0.5 * df
    f = freq / fe
    
    z, p, k = iirfilter(corners, f, btype='lowpass', ftype='butter', output='zpk')
    sos = zpk2sos(z, p, k)
    if zerophase:
        firstpass = sosfilt(sos, data)
        return sosfilt(sos, firstpass[::-1])[::-1]
    else:
        return sosfilt(sos, data)

In [10]:
# 速度微分 -> 加速度
# 100.0: sampling_rate
def v_to_a(z, n, e, sample_rate):
    d_z = np.gradient(z, 1.0/sample_rate)
    d_n = np.gradient(n, 1.0/sample_rate)
    d_e = np.gradient(e, 1.0/sample_rate)

    return d_z, d_n, d_e

In [11]:
# 加速度積分 -> 速度
# 100.0: sampling_rate
def a_to_v(z, n, e, sample_rate):
    i_z = integrate.cumtrapz(z)/sample_rate
    i_n = integrate.cumtrapz(n)/sample_rate
    i_e = integrate.cumtrapz(e)/sample_rate
    
    return i_z, i_n, i_e

In [ ]:
# 將舊制的 location 記錄方式轉成新制
def convert_to_new_location(old):
    # 地表
    if old == '01':
        return '10'
    # 井下
    elif old == '02':
        return '00'
    # 海底
    else:
        return '20'

In [ ]:
# 將舊制的儀器記錄方式轉成新制 channel 格式
def convert_instrument_to_channel(instrument):
    if instrument == 'FBA':
        return 'HL'
    elif instrument == 'SP':
        return 'EH'
    elif instrument == 'BB':
        return 'HH'

In [13]:
# 利用 json 來檢查是否有多個事件在同個 trace 當中
def check_multiple_event(filename):
    # filename: /mnt/nas2/CWBSN_modify_time/2012/13160139(0).json
    
    event = filename.split('(')[0]
    
    files = glob.glob(event+'*.json')
    
    return len(files)

In [16]:
# 根據測站名稱去找 station_code (CWBSN, TSMIP)
def find_station_code(station, query, network):
    if network == 'CWBSN':
        return station.loc[station['CWBSN_code']==query]['Station_Code'].iloc[0]
    elif network == 'TSMIP':
        return station.loc[station['TSMIP_code']==query]['Station_Code'].iloc[0]

In [17]:
# 根據 station_code, 資料記錄時間來找到測站儀器的座標與轉換量
def find_coord_and_factor(usage, sensor, query, time, instrument):
    candidate = usage.loc[usage['station_code']==query]
    time = int(time)
    
    candidate_sensor = {}
    for c in range(len(candidate)):
        if time >= candidate['start_date'].iloc[c] and time <= candidate['end_date'].iloc[c]:
            candidate_sensor[str(c)] = candidate['sensor_code'].iloc[c]
    
    # 有多個 rows 符合，進一步比對 intrument 跟 sensor 裡面記錄的儀器種類
    if len(candidate_sensor) > 1:
        for k, v in candidate_sensor.items():
            candidate_intrument = sensor.loc[sensor['Simple_Code']==v]
            
            for c in range(len(candidate_intrument)):
                if candidate_intrument['Type'].iloc[c] == instrument:
                    if math.isnan(candidate['Z_factor'].iloc[c]):
                        continue
                        
                    c = int(k)
                    print(v)
                    return [candidate['Z_factor'].iloc[c], candidate['N_factor'].iloc[c], candidate['E_factor'].iloc[c]], \
                      candidate['Ins_longitude'].iloc[c], candidate['Ins_latitude'].iloc[c], candidate['Ins_elevation'].iloc[c]
                
    elif len(candidate_sensor) == 0:
        return None, None, None
    else:
        c = int(list(candidate_sensor.keys())[0])
        return [candidate['Z_factor'].iloc[c], candidate['N_factor'].iloc[c], candidate['E_factor'].iloc[c]], \
            candidate['Ins_longitude'].iloc[c], candidate['Ins_latitude'].iloc[c], candidate['Ins_elevation'].iloc[c]